In [29]:
from google.cloud import bigquery
import time
import datetime as dt
import random

# Table schema
table_schema = [
    bigquery.SchemaField('time', 'DATETIME', mode='REQUIRED'),
    bigquery.SchemaField('bid', 'FLOAT', mode='REQUIRED'),
    bigquery.SchemaField('offer', 'FLOAT', mode='REQUIRED'),
    bigquery.SchemaField('bid_depth', 'INTEGER', mode='NULLABLE'),
    bigquery.SchemaField('offer_depth', 'INTEGER', mode='NULLABLE'),
    bigquery.SchemaField('bid_depth_total', 'INTEGER', mode='NULLABLE'),
    bigquery.SchemaField('appended', 'DATETIME', mode='REQUIRED'),
]

class BigQuerySession(object):
    """BigQuery session for a project."""
    
    def __init__(self, CREDS_DIR, project_id='quantum-fusion-233713'):
        self.client = bigquery.Client.from_service_account_json(CREDS_DIR)
        self.project_id = project_id
        
    def create_table(self, table_name, schema, dataset_name='netfonds_data'):
        """Create new table in dataset.

        Parameters:
            dataset_name: str
            client: google.cloud.bigquery.client.Client
            schema: list of bigquery.SchemaField
        """
        dataset_ref = self.client.dataset(dataset_name)
        table_ref = dataset_ref.table(table_name)
        table = bigquery.Table(table_ref, schema=schema)
        table = self.client.create_table(table)  # API request

        assert table.table_id == table_name
        
    def get_tables(self, dataset_name='netfonds_data'):
        query_job = self.client.query("""
            SELECT *
            FROM `{}.{}.__TABLES__`
            """.format(self.project_id, dataset_name))
        
        return query_job.result()
        
    def delete_table(self, table_name, dataset_name='netfonds_data'):
        table_ref = self.client.dataset(dataset_name).table(table_name)
        self.client.delete_table(table_ref)  # API request
        

BQS = BigQuerySession(CREDS_DIR='../data/credentials/quantum-fusion-233713-5ec9fa7a9e62.json')

In [5]:
# For this sample, the table must already exist and have a defined schema
dataset_id = 'netfonds_data'  # replace with your dataset ID
table_id = 'test'  # replace with your table ID
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)  # API request

for _ in range(10):
    ex_dt = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
    rows_to_insert = [
        {"DATE": ex_dt, "TEST": chr(random.randint(0, 1000))}
    ]
    errors = client.insert_rows(table, rows_to_insert)  # API request
    assert errors == []
    time.sleep(0.2)